# Setup printing functions and load data

In [8]:
using ClobberingReload
@ausing GERDADeepLearning

function print_rej(events)
    N = eventcount(events)
    
    AoE_set = filter(events, :AoE_class, x->x!=-1)
    N_AoE = eventcount(AoE_set)
    rej_AoE = eventcount(filter(AoE_set, [:AoE, :AoE_class], (AoE, cla) -> (AoE<0)&&(cla==1)))
    rej_PNN_AoE = sum(AoE_set[:psd_class])
    
    ANN_set = filter(events, :ANN_mse_class, x->x!=-1)
    N_ANN = eventcount(ANN_set)
    rej_ANN = eventcount(ANN_set[:ANN_mse_class=>1])
    rej_PNN_ANN = sum(ANN_set[:psd_class])
    
    rej = rej_PNN_AoE + rej_PNN_ANN
    
    if N_AoE > 0
        println(" ---- BEGes ----")
        println("PNN rejected $rej_PNN_AoE ($(Int64(round(100*rej_PNN_AoE/N_AoE)))\\%) out of $N_AoE")
        println("A/E rejected $rej_AoE ($(Int64(round(100*rej_AoE/N_AoE)))\\%) out of $N_AoE")
    end
    if N_ANN > 0
        println(" ---- Coax  ----")
        println("PNN rejected $rej_PNN_ANN ($(Int64(round(100*rej_PNN_ANN/N_ANN)))\\%) out of $N_ANN")
        println("ANN rejected $rej_ANN ($(Int64(round(100*rej_ANN/N_ANN)))\\%) out of $N_ANN")
    end
    println(" ----Combined----")
    println("Two-stage rejected $rej ($(Int64(round(100*rej/N)))\\%) out of $N")
    println("Current combined rejected $(rej_ANN+rej_AoE) ($(Int64(round(100*(rej_ANN+rej_AoE)/N)))\\%) out of $N")
    
    println()
    println("    set & $N & $N_AoE & $rej_PNN_AoE ($(Int64(round(100*rej_PNN_AoE/N_AoE)))\\%) & $N_ANN & $rej_PNN_ANN ($(Int64(round(100*rej_PNN_ANN/N_ANN)))\\%) & $rej ($(Int64(round(100*rej/N)))\\%) \\Tstrut \\\\")
    println("    & & & $rej_AoE ($(Int64(round(100*rej_AoE/N_AoE)))\\%) & & $rej_ANN ($(Int64(round(100*rej_ANN/N_ANN)))\\%) & $(rej_ANN+rej_AoE) ($(Int64(round(100*(rej_ANN+rej_AoE)/N)))\\%)  \\Bstrut \\\\")
end

function print_eff(events)
    N = eventcount(events)
    pass = N - sum(events[:psd_class])
    N_AoE = eventcount(filter(events, :AoE_class, x->x!=-1))
    pass_AoE = N_AoE - eventcount(filter(events, [:AoE, :AoE_class], (AoE, cla) -> (AoE<0)&&(cla==1)))
    N_ANN = eventcount(filter(events[:AoE_class=>-1], :ANN_mse_class, x->x!= -1))
    pass_ANN = eventcount(events[:AoE_class=>-1, :ANN_mse_class=>0])
    println("Two-stage eff: $(round(100*pass/N, 1))\\pm $(round(100*sqrt(pass)/N, 1)) \\% ($pass / $N)")
    println("A/E       eff: $(round(100*pass_AoE/N_AoE, 1))\\pm $(round(100*sqrt(pass_AoE)/N_AoE, 1)) \\% ($pass_AoE / $N_AoE)")
    println("ANN       eff: $(round(100*pass_ANN/N_ANN, 1))\\pm $(round(100*sqrt(pass_ANN)/N_ANN, 1)) \\% ($pass_ANN / $N_ANN)")
end

env = DLEnv();
data = get(env, "classified")
println("Total events: $(eventcount(data))")

BEGes = detectors(env, "BEGe")
BEGe_data = filter(data, :detector_name, det -> det ∈ BEGes)
filter!(BEGe_data, :AoE_class, x->x!=-1)
println("Valid events: $(eventcount(data))")

Total events: 175593
Valid events: 169764


# Data selection

Execute the following cell to select only BEGe detectors

In [6]:
BEGes = detectors(env, "BEGe", "used")
data = filter(data, :detector_name, det -> det ∈ BEGes)
println("There are $(eventcount(data[:AoE_class=>-1])) from BEGes that are not evaluated")

There are 0 from BEGes that are not evaluated


Execute the following cell to select only semi-coaxial detectors

In [9]:
coax = detectors(env, "coax")
data = filter(data, :detector_name, det -> det ∈ coax)
println("There are $(eventcount(filter(data, :AoE_class, x->x!=-1))) from BEGes that are not evaluated")

There are 0 from BEGes that are not evaluated


# Data sets

Muon set

In [169]:
muons = data[:isMuVetoed=>1]
print_rej(muons)

 ---- BEGes ----
PNN rejected 439 (73\%) out of 605
A/E rejected 243 (40\%) out of 605
 ---- Coax  ----
PNN rejected 152 (42\%) out of 361
ANN rejected 136 (38\%) out of 361
 ----Combined----
Two-stage rejected 591 (61\%) out of 966
Current combined rejected 379 (39\%) out of 966

    set & 966 & 605 & 439 (73\%) & 361 & 152 (42\%) & 591 (61\%) \Tstrut \\
    & & & 243 (40\%) & & 136 (38\%) & 379 (39\%)  \Bstrut \\


LAr set

In [170]:
LAr = data[:isLArVetoed=>1]
print_rej(LAr)

 ---- BEGes ----
PNN rejected 34428 (55\%) out of 62705
A/E rejected 16440 (26\%) out of 62705
 ---- Coax  ----
PNN rejected 12504 (37\%) out of 34090
ANN rejected 13309 (39\%) out of 34090
 ----Combined----
Two-stage rejected 46932 (48\%) out of 96795
Current combined rejected 29749 (31\%) out of 96795

    set & 96795 & 62705 & 34428 (55\%) & 34090 & 12504 (37\%) & 46932 (48\%) \Tstrut \\
    & & & 16440 (26\%) & & 13309 (39\%) & 29749 (31\%)  \Bstrut \\


Anti-coincidence set

In [171]:
ac = filter(data, :multiplicity, m->m>1)
print_rej(ac)

 ---- BEGes ----
PNN rejected 6498 (58\%) out of 11157
A/E rejected 3139 (28\%) out of 11157
 ---- Coax  ----
PNN rejected 1429 (36\%) out of 3943
ANN rejected 1424 (36\%) out of 3943
 ----Combined----
Two-stage rejected 7927 (52\%) out of 15100
Current combined rejected 4563 (30\%) out of 15100

    set & 15100 & 11157 & 6498 (58\%) & 3943 & 1429 (36\%) & 7927 (52\%) \Tstrut \\
    & & & 3139 (28\%) & & 1424 (36\%) & 4563 (30\%)  \Bstrut \\


2vbb set

In [172]:
doublebeta = filter(data[:isMuVetoed=>0, :multiplicity=>1, :isLArVetoed=>0, :FailedPreprocessing=>0], :E, E->(E>1000)&&(E<1300))
print_rej(doublebeta)

 ---- BEGes ----
PNN rejected 307 (15\%) out of 2114
A/E rejected 240 (11\%) out of 2114
 ---- Coax  ----
PNN rejected 382 (21\%) out of 1852
ANN rejected 428 (23\%) out of 1852
 ----Combined----
Two-stage rejected 689 (17\%) out of 3966
Current combined rejected 668 (17\%) out of 3966

    set & 3966 & 2114 & 307 (15\%) & 1852 & 382 (21\%) & 689 (17\%) \Tstrut \\
    & & & 240 (11\%) & & 428 (23\%) & 668 (17\%)  \Bstrut \\


Alpha set

In [173]:
alpha = filter(data[:isMuVetoed=>0, :multiplicity=>1, :isLArVetoed=>0, :FailedPreprocessing=>0], :E, E->(E>3000)&&(E<9999))
print_rej(alpha)

 ---- BEGes ----
PNN rejected 130 (56\%) out of 234
A/E rejected 0 (0\%) out of 234
 ---- Coax  ----
PNN rejected 171 (25\%) out of 689
ANN rejected 159 (23\%) out of 689
 ----Combined----
Two-stage rejected 301 (33\%) out of 923
Current combined rejected 159 (17\%) out of 923

    set & 923 & 234 & 130 (56\%) & 689 & 171 (25\%) & 301 (33\%) \Tstrut \\
    & & & 0 (0\%) & & 159 (23\%) & 159 (17\%)  \Bstrut \\


Region of interest

In [174]:
roi = filter(data[:FailedPreprocessing=>0], :E, E->(E>2038.5-50)&&(E<2039.5+50))
print_rej(roi)

 ---- BEGes ----
PNN rejected 6 (67\%) out of 9
A/E rejected 4 (44\%) out of 9
 ---- Coax  ----
PNN rejected 4 (31\%) out of 13
ANN rejected 3 (23\%) out of 13
 ----Combined----
Two-stage rejected 10 (45\%) out of 22
Current combined rejected 7 (32\%) out of 22

    set & 22 & 9 & 6 (67\%) & 13 & 4 (31\%) & 10 (45\%) \Tstrut \\
    & & & 4 (44\%) & & 3 (23\%) & 7 (32\%)  \Bstrut \\


# Individual Events in ROI

Select only events with multiplicity 1.

In [175]:
acroi = sort(filter(data[:multiplicity=>1, :FailedPreprocessing=>0], :E, E->(E>2038.5-50)&&(E<2039.5+50)), :E)
eventcount(acroi)

12

In [8]:
println("Energy & Detector & Recorded & Muon & LAr & PSD & Classification \\\\")
println("\\hline")
for i in 1:eventcount(acroi)
    det = lookup_property(acroi, i, data, :detector_name)
    time = Dates.unix2datetime(acroi[:timestamp][i])
    date = "$(Dates.monthname(time)) $(Dates.day(time)), $(Dates.year(time))"
    muon = acroi[:isMuVetoed][i] == 1 ? "Yes" : "No"
    LAr = acroi[:isLArVetoed][i] == 1 ? "Yes" : "No"
    PNN = acroi[:psd_class][i] == 1 ? "Yes" : "No"
    if startswith(det, "GD")
        PSD = ((acroi[:AoE_class][i] == 1)&&(acroi[:AoE][i] < 0)) ? "Yes" : "No"
    else
        PSD = acroi[:ANN_mse_class] == 1 ? "Yes" : "No"
    end
    energy = Int64(round(acroi[:E][i]))
    println("$energy keV & $det & $date & $muon & $LAr & $PSD & $PNN \\\\")
end

Energy & Detector & Recorded & Muon & LAr & PSD & Classification \\
\hline
1989 keV & GD61C & April 5, 2016 & No & Yes & No & Yes \\
1991 keV & ANG5 & April 26, 2016 & No & Yes & No & No \\
1992 keV & ANG3 & December 31, 2015 & No & Yes & No & No \\
1994 keV & ANG4 & February 10, 2016 & No & No & No & No \\
1997 keV & GD89A & May 4, 2016 & No & No & No & No \\
2009 keV & ANG2 & April 26, 2016 & No & No & No & Yes \\
2049 keV & GD02B & January 3, 2016 & No & Yes & Yes & Yes \\
2062 keV & GD61C & May 18, 2016 & No & Yes & Yes & Yes \\
2064 keV & ANG3 & March 28, 2016 & No & No & No & No \\
2065 keV & GD89B & January 1, 2016 & No & Yes & No & No \\
2074 keV & RG1 & May 1, 2016 & No & No & No & No \\
2082 keV & ANG5 & May 29, 2016 & No & Yes & No & Yes \\
